# SysML v2 Test Suite

This can notebook can be cleared using menu `Edit --> Clear All Outputs` and then all cells can be
run using menu `Run --> Run All Cells`.

**WARNING**: This file is generated, all modifications here will be ignored.


## Base Example Directory: ../../sysml/src/examples/Vehicle Example

## File: VehicleDefinitions.sysml

In [1]:
package VehicleDefinitions {
	doc
	/*
	 * Example vehicle definitions model.
	 */

	import ScalarValues::*;
	import Quantities::*;
	import MeasurementReferences::*;
	import ISQ::*;
	import SI::*;
	
	/* PART DEFINITIONS */
	
	part def Vehicle;	
	part def Transmission;	
	part def AxleAssembly;
	part def Axle {
		port leftMountingPoint: AxleMountIF;
		port rightMountingPoint: AxleMountIF;
	}
	part def Wheel {
		port hub: WheelHubIF;
	}
	part def Lugbolt {
		attribute tighteningTorque :> ISQ::torque;
	}
	
	/* PORT DEFINITIONS */
	
	port def DriveIF { 
		in driveTorque :> ISQ::torque;
	}
	
	port def AxleMountIF { 
		out transferredTorque :> ISQ::torque;
	}
	
	port def WheelHubIF { 
		in appliedTorque :> ISQ::torque;
	}
	
	/* INTERFACE DEFINITIONS */
	
	interface def Mounting {
		doc /* The definition of the interface for mounting a Wheel to an Axle. */
		end axleMount: AxleMountIF;
		end hub: WheelHubIF;
		
		flow axleMount.transferredTorque to hub.appliedTorque;
	}
}

Package VehicleDefinitions (bb48ba0b-afd4-458d-8c09-6f6faef7f8ab)


## File: VehicleUsages.sysml

In [2]:
package VehicleUsages {
	doc
	/*
	 * Example usages of elements from the vehicle definitions model.
	 */

	import SI::N;
	import SI::m;
	import VehicleDefinitions::*;
	import ScalarFunctions::*;

	/* VALUES */	 
	T1 = 10.0 [N * m];
	T2 = 20.0 [N * m];
	
	/* PARTS */	
	part narrowRimWheel: Wheel {
		doc /* Narrow-rim wheel configuration with 4 to 5 lugbolts. */

		part lugbolt: Lugbolt[4..5];
	}
	
	part wideRimWheel: Wheel {
		doc /* Wide-rim wheel configuration with 4 to 6 lugbolts. */	

		part lugbolt: Lugbolt[4..6];
	}

	part vehicle_C1: Vehicle {
		doc /* Basic Vehicle configuration showing a part hierarchy. */

		part frontAxleAssembly: AxleAssembly {
			part frontWheel[2] subsets narrowRimWheel {
				part redefines lugbolt[4] {
					attribute redefines tighteningTorque = T1;
				}
			}
			part frontAxle: Axle;
		}		
		part rearAxleAssembly: AxleAssembly {
			part rearWheel[2] subsets wideRimWheel {
				part redefines lugbolt[6] {
					attribute redefines tighteningTorque = T2;
				}
			}
			part rearAxle: Axle;			
		}
	}
	
	part vehicle_C2 subsets vehicle_C1 {
		doc /* Specialized configuration with part-specific ports. */

		part redefines frontAxleAssembly {
			part leftFrontWheel subsets frontWheel = frontWheel[1];
			part rightFrontWheel subsets frontWheel = frontWheel[2];
			
			interface leftFrontMount: Mounting connect 
				frontAxle.leftMountingPoint to leftFrontWheel.hub;
				
			interface rightFrontMount: Mounting connect 
				frontAxle.rightMountingPoint to rightFrontWheel.hub;
		}
		
		part rearAxleAssembly redefines vehicle_C1::rearAxleAssembly {
			part leftRearWheel subsets rearWheel = rearWheel[1];
			part rightRearWheel subsets rearWheel = rearWheel[2];

			interface leftRearMount: Mounting connect 
				rearAxle.leftMountingPoint to leftRearWheel.hub;
				
			interface rightRearMount: Mounting connect 
				rearAxle.rightMountingPoint to rightRearWheel.hub;
		}		
	}
	
	part vehicle_C3 subsets vehicle_C2 {
		doc /* Further specialized configuration with a connection to a deeply-nested port. */

		
		part transmission: Transmission {
			port drive: ~DriveIF;
		}
		
		part redefines rearAxleAssembly {
			part redefines rearAxle {
				port drive: DriveIF;
			}
		}
		
		interface driveShaft connect 
			transDrive :> transmission.drive to axleDrive :> rearAxleAssembly.rearAxle.drive {
			flow transDrive.driveTorque to axleDrive.driveTorque;
		}		
	}
}

Package VehicleUsages (d989c9e1-f3dc-4876-b21f-8c8d8ef6ffd6)


## File: VehicleIndividuals.sysml

In [3]:
package VehicleIndividuals {
	import VehicleUsages::*;
	import Time::DateTime;
	
	package IndividualDefinitions {

		individual part def Vehicle1 :> Vehicle {
			doc
			/*
			 * This is an individual Vehicle with a mass of 1800 kg.
			 */
			
			attribute redefines mass = 1800 [kg];
		}
		
		individual part def Vehicle2 :> Vehicle {
			doc
			/*
			 * This is an individual Vehicle with a mass of 1700 kg.
			 */
		
			attribute redefines mass = 1700 [kg];
		}
		
		individual part def AxleAssembly1 :> AxleAssembly;
		
		individual part def Wheel1 :> Wheel;
		individual part def Wheel2 :> Wheel;
	}
	
	package IndividualSnapshots {
		import IndividualDefinitions::*;
	
		attribute t0: DateTime;
		attribute t1: DateTime;
		
		snapshot part vehicle1_t0: Vehicle1 {
			doc
			/*
			 * This is a snapshot of Vehicle1 at time t0;
			 */
		
			// Note: Timestamping of snapshots is not yet formalized.
			attribute t = t0;
		}
		
		timeslice part vehicle1_t0_t1: Vehicle1 {
			doc
			/*
			 * This is a time slice of Vehicle1 starting at snapshot vehicle1_t0 
			 * (time t0) and ending at time t1.
			 */
		
			snapshot :>> start = vehicle1_t0;
			snapshot :>> done {
				attribute t = t1;
			}
		}	
	}
	
	package IndividualConfigurations {
		import IndividualSnapshots::*;
	
		individual part vehicle1_C2: Vehicle1 :> vehicle_C2 {
			doc
			/*
			 * This asserts that for some portion of its lifetime, Vehicle1 conforms
			 * to the configuration vehicle_C2;
			 */
		}	
			
		snapshot part vehicle1_C2_t0 :> vehicle1_C2, vehicle1_t0 {
			doc
			/*
			 * This is a snapshot of Vehicle1 in configuration vehicle1_C2 at time t0.
			 */
		
			snapshot axleAssembly1_t0: AxleAssembly1 :>> frontAxleAssembly {
				doc
				/*
				 * frontAxleAssembly is a feature of vehicle1_C2.
				 */
			
				snapshot leftFrontWheel_t0: Wheel1 :>> leftFrontWheel {
					doc
					/*
					 * This asserts that Wheel1 is the leftFrontWheel of vehicle_C2_t0
					 * (leftFrontWheel is a feature of vehicle_C2::frontAxleAssembly).
					 */
				}
			}
		}
		
		snapshot part vehicle1_C2_t1 :> vehicle1_C2, vehicle1_t0_t1.done {
			doc
			/*
			 * This is a snapshot of Vehicle1 in configuration vehicle_C2 at time t1.
			 */
		
			snapshot axleAssembly1_t1: AxleAssembly1 :>> frontAxleAssembly {
				snapshot rightFrontWheel_t1: Wheel1 :>> rightFrontWheel {
					doc
					/*
					 * This asserts that Wheel1 is the rightFrontWheel of vehicle_C2_t1.
					 */
				}
			}
		}	
	}
}

Package VehicleIndividuals (2347658e-5ec1-43ce-80c8-fef3c873ab83)
